In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
! git clone https://github.com/dleve123/topics-in-nlp-repro-project.git

Cloning into 'topics-in-nlp-repro-project'...
remote: Enumerating objects: 353, done.
remote: Counting objects: 100% (353/353), done.
remote: Compressing objects: 100% (229/229), done.
remote: Total 353 (delta 167), reused 272 (delta 103), pack-reused 0
Receiving objects: 100% (353/353), 2.16 MiB | 3.88 MiB/s, done.
Resolving deltas: 100% (167/167), done.


In [3]:
%cd topics-in-nlp-repro-project
! git checkout main

/content/topics-in-nlp-repro-project
Already on 'main'
Your branch is up to date with 'origin/main'.


In [4]:
! git fetch
! git reset --hard origin/main

HEAD is now at b16ea2a Update README.md


In [5]:
%%capture
! pip install -r requirements.txt

In [ ]:
! python eval_bert_score.py xsum/test facebook-bart-large-xsum

Using custom data configuration default
Reusing dataset xsum (/root/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934)
100% 3/3 [00:00<00:00, 540.71it/s]
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
P=0.926325 R=0.917188 F=0.921628
calculated 11334 scores, saving to storage..


## Validate that we get the same results for paper-published data

In [ ]:
gen_sums = []
gt_sums = []
corrected_sums = []
source_docs = []

with open("./validate_results/bart.part.txt", "r") as f:
    for line in f:
        gen_sums.append(line.strip().lower())

with open("./validate_results/target.part.txt", "r") as f:
    for line in f:
        gt_sums.append(line.strip().lower())

with open("./validate_results/corrected.part.txt", "r") as f:
    for line in f:
        corrected_sums.append(line.strip().lower())

with open("./validate_results/source.part.txt", "r") as f:
    for line in f:
        source_docs.append(line.strip().lower())

In [ ]:
import numpy as np
from rouge_score import rouge_scorer

rouge_l_scorer = rouge_scorer.RougeScorer(
    ['rougeLsum'],
    use_stemmer=True
)

bart_rouge_scores = []
corrected_rouge_scores = []
for gt_summary, bart_summary, corrected_summary in zip(gt_sums, gen_sums, corrected_sums):
    bart_rouge_scores.append(
        rouge_l_scorer.score(gt_summary, bart_summary)['rougeLsum'].fmeasure
    )
    corrected_rouge_scores.append(
        rouge_l_scorer.score(gt_summary, corrected_summary)['rougeLsum'].fmeasure
    )

paper_bart = 38.63
paper_corrected = 36.62
our_bart = np.mean(bart_rouge_scores) * 100
our_corrected = np.mean(corrected_rouge_scores) * 100

print("Rouge Scores")
print(f"BART-large, paper: {paper_bart}, ours: {our_bart:.2f}, diff: {(paper_bart - our_bart):.2f}")
print(f"Corrected, paper: {paper_corrected}, ours: {our_corrected:.2f}, diff: {(paper_corrected - our_corrected):.2f}")


Rouge Scores
BART-large, paper: 38.63, ours: 37.80, diff: 0.83
Corrected, paper: 36.62, ours: 35.81, diff: 0.81


In [ ]:
# Backwards-compatibility for bert-score
! pip install transformers==3.5.1
! pip install bert-score==0.3.6
! pip install torch==1.4.0

In [ ]:
from bert_score import score
import numpy as np

def evaluate_bert_score(gt_summaries, generated_summaries):
    (P, R, F), hashname = score(
        # confirmed that order of args and lower case doesnt matter
        # rescale with baseline does not work
        [x for x in generated_summaries],   
        [x for x in gt_summaries],
        lang="en", 
        return_hash=True,
        # rescale_with_baseline=True
    )
    print("hashname", hashname)
    print(f"P={P.mean().item():.6f} R={R.mean().item():.6f} F={F.mean().item():.6f}")
    return F.tolist()

bart_bert_scores = evaluate_bert_score(gt_sums, gen_sums)
corrected_bert_scores = evaluate_bert_score(gt_sums, corrected_sums)

paper_bart = 91.61
paper_corrected = 91.10
our_bart = np.mean(bart_bert_scores) * 100
our_corrected = np.mean(corrected_bert_scores) * 100

print("BERT Scores")
print(f"BART-large, paper: {paper_bart}, ours: {our_bart:.2f}, diff: {(paper_bart - our_bart):.2f}")
print(f"Corrected, paper: {paper_corrected}, ours: {our_corrected:.2f}, diff: {(paper_corrected - our_corrected):.2f}")


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,


hashname roberta-large_L17_no-idf_version=0.3.6(hug_trans=3.5.1)-rescaled
P=0.480569 R=0.459697 F=0.470328


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
     a = 1.0 * np.array(data)
     n = len(a)
     m, se = np.mean(a), scipy.stats.sem(a)
     h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
     return m, m-h, m+h
    
mean_confidence_interval(bart_bert_scores)

(0.9106051067248085, 0.9091343403859644, 0.9120758730636527)

In [ ]:
mean_confidence_interval(corrected_bert_scores)

(0.9043894830523738, 0.9030304781051935, 0.905748487999554)

In [ ]:
def draw_bs_replicates(data,func,size):
    """creates a bootstrap sample, computes replicates and returns replicates array"""
    # Create an empty array to store replicates
    bs_replicates = np.empty(size)
    
    # Create bootstrap replicates as much as size
    for i in range(size):
        # Create a bootstrap sample
        bs_sample = np.random.choice(data,size=len(data))
        # Get bootstrap replicate and append to bs_replicates
        bs_replicates[i] = func(bs_sample)
    
    return bs_replicates

bs_replicates = draw_bs_replicates(bart_bert_scores,np.mean,15000)

In [ ]:
conf_interval = np.percentile(bs_replicates,[2.5,97.5])
conf_interval

array([0.90911933, 0.91208589])

In [ ]:
# Reset deps after computing bert-score
! pip install torch==1.11.0
! pip install transformers==4.17.0

# Utils

In [6]:
from rouge_score import rouge_scorer
import scipy.stats
from bert_score import score
import numpy as np

def compute_rouge_scores(gt_sums, gen_sums, metric="rougeL"):
  rouge_l_scorer = rouge_scorer.RougeScorer(
      ['rougeL'],
      use_stemmer=True
  )

  rouge_scores = []
  for gt_sum, gen_sum in zip(gt_sums, gen_sums):
      rouge_scores.append(
          rouge_l_scorer.score(gt_sum, gen_sum)['rougeL'].fmeasure
      )
  return rouge_scores

def compute_bert_scores(gt_summaries, generated_summaries):
    (P, R, F), hashname = score(
        # confirmed that order of args and lower case doesnt matter
        # rescale with baseline does not work
        [x for x in generated_summaries],   
        [x for x in gt_summaries],
        lang="en", 
        return_hash=True,
        # rescale_with_baseline=True
    )
    print("BERT-score hashname", hashname)
    return F.tolist()

def mean_confidence_interval(data, confidence=0.95):
     a = 1.0 * np.array(data)
     n = len(a)
     m, se = np.mean(a), scipy.stats.sem(a)
     h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
     return m-h, m, m+h

# Final

In [7]:
%%capture
from eval_bert_score import load_data

gt_sums, bart_large_sums, sum_ids = load_data(
    "xsum", 
    "test", 
    "facebook-bart-large-xsum"
)

### BART-large

In [8]:
import numpy as np
rouge_scores = compute_rouge_scores(
    gt_sums,
    bart_large_sums,
    metric="rougeL"
)
bert_scores = compute_bert_scores(
    gt_sums,
    bart_large_sums
)

print("ROUGE-L", mean_confidence_interval(rouge_scores))
print("BERT", mean_confidence_interval(bert_scores))

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERT-score hashname roberta-large_L17_no-idf_version=0.3.7(hug_trans=4.17.0)
ROUGE-L (0.36909134897464074, 0.3722522387559257, 0.3754131285372107)
BERT (0.9211224615764596, 0.9216275771911211, 0.9221326928057826)


## +Corrected

In [9]:
import json
# Load corrected summaries
corr_sums = []
with open("/content/gdrive/MyDrive/CS6741/replication/data/corrected/train_v1_final_280k.jsonl", "r") as f:
  for line in f:
    obj = json.loads(line)
    corr_sums.append((
        obj["corrected"],
        obj["source"]
    ))
len(corr_sums)

1946

In [10]:
from datasets import load_dataset
xsum_test = load_dataset("xsum")["test"]
xsum_test_source_to_id = {x["document"]: x["id"] for x in xsum_test}
xsum_doc_by_id = {x["id"]: x for x in xsum_test}

Using custom data configuration default
Reusing dataset xsum (/root/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934)


  0%|          | 0/3 [00:00<?, ?it/s]

### Rouge & BERT-score for corrected summaries (corrector)

In [19]:
n = 0
corr_sums_with_gt = []
corr_sums_gt = []
corr_sum_by_id = {}

corr_sums_with_gt_full = []
for sum, source in corr_sums:
  if source in xsum_test_source_to_id:
    id = xsum_test_source_to_id[source]
    corr_sums_gt.append(xsum_doc_by_id[id]["summary"])
    corr_sums_with_gt.append(sum)
    n += 1
    corr_sum_by_id[id] = sum
print(n)

1766


In [12]:
import numpy as np
rouge_scores = compute_rouge_scores(
    corr_sums_gt,
    corr_sums_with_gt,
    metric="rougeL"
)
bert_scores = compute_bert_scores(
    corr_sums_gt,
    corr_sums_with_gt
)

print("ROUGE-L", mean_confidence_interval(rouge_scores))
print("BERT", mean_confidence_interval(bert_scores))

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERT-score hashname roberta-large_L17_no-idf_version=0.3.7(hug_trans=4.17.0)
ROUGE-L (0.36596905052894757, 0.3736233714162912, 0.38127769230363484)
BERT (0.9166923611232533, 0.9178685978949408, 0.9190448346666283)


### Rouge & BERT-score for corrected summaries (baseline)

In [13]:
bart_large_sums_by_id = {
    id: (sum, gt_sum) for id, sum, gt_sum in zip(sum_ids, bart_large_sums, gt_sums)
}
bart_large_subset_sums = []
bart_large_subset_gt = []
for id in corr_sum_ids:
  sum, gt_sum = bart_large_sums_by_id[id]
  bart_large_subset_sums.append(sum)
  bart_large_subset_gt.append(gt_sum)

In [14]:
import numpy as np
rouge_scores = compute_rouge_scores(
    bart_large_subset_gt,
    bart_large_subset_sums,
    metric="rougeL"
)
bert_scores = compute_bert_scores(
    bart_large_subset_gt,
    bart_large_subset_sums
)

print("ROUGE-L", mean_confidence_interval(rouge_scores))
print("BERT", mean_confidence_interval(bert_scores))

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERT-score hashname roberta-large_L17_no-idf_version=0.3.7(hug_trans=4.17.0)
ROUGE-L (0.3881612320079963, 0.3960413988179433, 0.40392156562789033)
BERT (0.923100832877441, 0.9243256134125546, 0.9255503939476681)


### Rouge & BERT-score for test set (corrector)

In [21]:
# Corrector sum if corrected, otherwise bart sum
bart_large_sums_by_id = {
    id: (sum, gt_sum) for id, sum, gt_sum in zip(sum_ids, bart_large_sums, gt_sums)
}
corrector_test_set_sums = []
corrector_test_set_sums_gt = []

for id, (bart_sum, gt_sum) in bart_large_sums_by_id.items():
  corrector_test_set_sums_gt.append(gt_sum)
  if id in corr_sum_by_id:
    corrector_test_set_sums.append(corr_sum_by_id[id])
  else:
    corrector_test_set_sums.append(bart_sum)

print(len(corrector_test_set_sums), len(corrector_test_set_sums_gt))

11334 11334


In [22]:
rouge_scores = compute_rouge_scores(
    corrector_test_set_sums_gt,
    corrector_test_set_sums,
    metric="rougeL"
)
bert_scores = compute_bert_scores(
    corrector_test_set_sums_gt,
    corrector_test_set_sums
)

print("ROUGE-L", mean_confidence_interval(rouge_scores))
print("BERT", mean_confidence_interval(bert_scores))

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERT-score hashname roberta-large_L17_no-idf_version=0.3.7(hug_trans=4.17.0)
ROUGE-L (0.36561717391819104, 0.36875918807732, 0.37190120223644896)
BERT (0.9201191512844124, 0.9206214812575183, 0.9211238112306243)
